In [2]:
import torch
modellist=[0,0,0,0]
modellist[0] = torch.hub.load("bryandlee/animegan2-pytorch","generator",pretrained="face_paint_512_v1")
modellist[1] = torch.hub.load("bryandlee/animegan2-pytorch","generator",pretrained="face_paint_512_v2")
modellist[2] = torch.hub.load("bryandlee/animegan2-pytorch","generator",pretrained="celeba_distill")
modellist[3] = torch.hub.load("bryandlee/animegan2-pytorch","generator",pretrained="paprika")
# modellist[4] = torch.hub.load("C:\\Users\\lenovo/.cache\\torch\\hub\\checkpoints\\generator_hayao.pth")

Using cache found in C:\Users\lenovo/.cache\torch\hub\bryandlee_animegan2-pytorch_main
Using cache found in C:\Users\lenovo/.cache\torch\hub\bryandlee_animegan2-pytorch_main
Using cache found in C:\Users\lenovo/.cache\torch\hub\bryandlee_animegan2-pytorch_main
Using cache found in C:\Users\lenovo/.cache\torch\hub\bryandlee_animegan2-pytorch_main


In [3]:
modellist[0].load_state_dict(torch.load('C:\\Users\\lenovo\\.cache\\torch\\hub\\checkpoints\\face_paint_512_v0.pt'))
modellist[3].load_state_dict(torch.load('C:\\Users\\lenovo\\.cache\\torch\\hub\\checkpoints\\face_paint_512_v2_0.pt'))

<All keys matched successfully>

In [4]:
for model in modellist:
    #print(model)
    model = model.eval().to('cuda')

In [9]:
print(modellist[0])

Generator(
  (block_a): Sequential(
    (0): ConvNormLReLU(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): Conv2d(3, 32, kernel_size=(7, 7), stride=(1, 1), bias=False)
      (2): GroupNorm(1, 32, eps=1e-05, affine=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (1): ConvNormLReLU(
      (0): ReflectionPad2d((0, 1, 0, 1))
      (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (2): GroupNorm(1, 64, eps=1e-05, affine=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (2): ConvNormLReLU(
      (0): ReflectionPad2d((1, 1, 1, 1))
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (2): GroupNorm(1, 64, eps=1e-05, affine=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  )
  (block_b): Sequential(
    (0): ConvNormLReLU(
      (0): ReflectionPad2d((0, 1, 0, 1))
      (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (2): GroupNorm(1, 128, eps=1e-05, affi

In [11]:
import cv2
import os
import numpy as np
from torchvision import transforms
from matplotlib import pyplot as plt
# name = input("请输入图片路径：")
head = "Baidu/people1"
# path = os.path.join(head,name)
# img = cv2.imread(path)
# img = cv2.resize(img,(800,800))
with torch.no_grad():
    # transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5])])
    # img = transform(img).to('cuda')
    # img = img
    # img = img.unsqueeze(dim=0)
    for pic in os.listdir(head):
        img = cv2.imread(os.path.join(head,pic))
        if img.shape[0]+img.shape[1]>3000:
            img = cv2.resize(img,(0,0),fx=0.5,fy=0.5)
        h,w,c = img.shape
        for i in range(3):
            temp = img[:,:,i]
            img[:,:,i] = 255*np.power(temp/255,0.8)#cv2.equalizeHist(temp)
        transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5])])
        img = transform(img).to('cuda')
        img = img.unsqueeze(dim=0)
        final = np.zeros((h,w,3))
        for i,model in enumerate(modellist):
            out = model(img)
            out = out.squeeze()
            out = out.permute(1,2,0)
            out = out.cpu().numpy()
            print(out.shape)
            out = 255*out#cv2.equalizeHist(np.uint8(temp*255))#255*np.power(out,0.5)
            #print(out)
            cv2.imwrite(os.path.join("outstyle",str(i+1),pic),out)#
            # if i==0:
            #     final += 0.3*out
            # if i==1:
            #     final += 0.2*out
            # if i==2:
            #     final += 0.35*out
            # if i==3:
            #     final += 0.05*out
            if i==0:
                final += 0.35*out
            if i==1:
                final += 0.3*out
            if i==2:
                final += 0.05*out
            if i==3:
                final += 0.3*out
        #print(final)
        cv2.imwrite(os.path.join("outstyle","add",pic),final)#str(i+1)

(1438, 960, 3)
(1438, 960, 3)
(1438, 960, 3)
(1438, 960, 3)
(800, 1280, 3)
(800, 1280, 3)
(800, 1280, 3)
(800, 1280, 3)
(1637, 1024, 3)
(1637, 1024, 3)
(1637, 1024, 3)
(1637, 1024, 3)
(1333, 1000, 3)
(1333, 1000, 3)
(1333, 1000, 3)
(1333, 1000, 3)
(360, 641, 3)
(360, 641, 3)
(360, 641, 3)
(360, 641, 3)


In [30]:
torch.cuda.empty_cache()

In [ ]:
test = cv2.imread("outstyle/3_hhtgan.jpg")
print(test)